# NYC Taxicab "Drift" Example

Author: shreyashankar

This notebook shows a toy example of a machine learning model that achieves similar performance on the train and evaluation sets but experiences performance "degradation" when simulating a "live" deployment.

In [1]:
from cuml.dask.ensemble import RandomForestClassifier
from cuml.metrics import roc_auc_score
from dask.array import from_array
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster
from progress import progress
from scipy import stats
from sklearn.metrics import f1_score

import dask_cudf
import dask.dataframe as dd
import pandas as pd

In [2]:
# Parameters

n_workers = 3

numeric_feat = [
    "pickup_weekday",
    "pickup_hour",
    'work_hours',
    "pickup_minute",
    "passenger_count",
    'trip_distance',
    'trip_time',
    'trip_speed'
]
categorical_feat = [
    "PULocationID",
    "DOLocationID",
    "RatecodeID",
]
features = numeric_feat + categorical_feat

EPS = 1e-7

## Initialize cluster

Using Saturn's predefined cluster setup.

In [3]:
progress('rf-rapids-dask-cluster-setup')
cluster = SaturnCluster(
    n_workers=n_workers, scheduler_size="medium", worker_size="g4dnxlarge"
)
client = Client(cluster)
client

[2021-02-11 02:14:09] INFO - dask-saturn | Cluster is ready
[2021-02-11 02:14:09] INFO - dask-saturn | Registering default plugins
[2021-02-11 02:14:09] INFO - dask-saturn | {'tcp://10.0.25.24:37137': {'status': 'repeat'}, 'tcp://10.0.4.201:38121': {'status': 'repeat'}, 'tcp://10.0.9.1:39615': {'status': 'repeat'}}


Client Scheduler: tcp://d-shrey-rapids-random-forest-85ae247e8459473fbfdc641eb0e7ecb2.main-namespace:8786 Dashboard: https://d-shrey-rapids-random-forest-85ae247e8459473fbfdc641eb0e7ecb2.community.saturnenterprise.io,Cluster Workers: 3 Cores: 12 Memory: 46.50 GB


## Create helper functions

In [4]:
def preprocess(df: dask_cudf.DataFrame, target_col: str, start_date: str = None, end_date: str = None) -> dask_cudf.DataFrame:
    """
    This function computes the target ('high_tip'), adds features, and removes unused features.
    Note that zero EDA or cleaning is performed here, whereas in the "real world" you should definitely
    inspect and clean the data. If a start or end date is specified, any entries outside of these bounds
    will be dropped from the dataframe.
    
    Args:
        df: dask dataframe representing data
        target_col: column name of the target (must be in df)
        start_date (optional): minimum date in the resulting dataframe
        end_date (optional): maximum date in the resulting dataframe
    
    Returns:
        dask_cudf: DataFrame representing the preprocessed dataframe
    """
    # Basic cleaning
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    if start_date:
        df = df[df.tpep_dropoff_datetime.astype('str') >= start_date]
    if end_date:
        df = df[df.tpep_dropoff_datetime.astype('str') <= end_date]

    # add target
    df["tip_fraction"] = df.tip_amount / df.fare_amount
    df[target_col] = df["tip_fraction"] > 0.2

    # add features
    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute
    df["work_hours"] = (df.pickup_weekday >= 0) & (df.pickup_weekday <= 4) & (df.pickup_hour >= 8) & (df.pickup_hour <= 18)
    df['trip_time'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.seconds
    df['trip_speed'] = df.trip_distance / (df.trip_time + EPS)

    # drop unused columns
    df = df[['tpep_dropoff_datetime'] + features + [target_col]]
    df[features + [target_col]] = df[features + [target_col]].astype("float32").fillna(-1.0)

    # convert target to int32 for efficiency (it's just 0s and 1s)
    df[target_col] = df[target_col].astype("int32")

    return df.reset_index(drop=True)

def f1_streaming(df: dask_cudf.DataFrame, target_col: str, pred_col: str) -> dask_cudf.Series:
    """
    Computes rolling precision and recall columns
    F1 = 2 * (precision * recall) / (precision + recall)

    Precision: of the rows we predicted true, how many were true?
    Recall: of all the trues, how many did we predict to be true?
    
    Args:
        df: dask dataframe
        target_col: column name of the target (must be in df)
        pred_col: column name of the prediction (must be in df)
    
    Returns:
        dask_cudf: Series representing the cumulative F1 score
    """
    df = df.sort_values(by=['tpep_dropoff_datetime'], ascending=True)
    numerator = (df['prediction'] & df[target_col]).cumsum()
    precision_denominator = df['prediction'].cumsum()
    recall_denominator = df[target_col].cumsum()
    precision = numerator / precision_denominator
    recall = numerator / recall_denominator
    return 2 * (precision * recall) / (precision + recall)

def get_daily_f1_score(partition):
    """
    """
    numerator = (partition[target_col] & partition['prediction']).sum()
    recall_denominator = partition[target_col].sum()
    precision_denominator = partition['prediction'].sum()
    precision = numerator / precision_denominator
    recall = numerator / recall_denominator
    f1_score = 2 * (precision * recall) / (precision + recall)
    partition['daily_f1'] = f1_score
    return partition.sort_values(by='tpep_dropoff_datetime', ascending=False).head(1)[['day', 'rolling_f1', 'daily_f1']]

## Load train data

The training window is all of January 2020 and accessible via a public s3 bucket.

In [5]:
taxi = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e9} GB")
taxi.head()

Num rows: 6405008, Size: 0.903424059 GB


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238.0,239.0,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239.0,238.0,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238.0,238.0,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238.0,151.0,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193.0,193.0,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [6]:
target_col = "high_tip"

taxi_train = preprocess(df=taxi, target_col=target_col)
print(f"Num rows: {len(taxi_train)}, Size: {taxi_train.memory_usage(deep=True).sum().compute() / 1e9} GB")

Num rows: 6382762, Size: 0.357434672 GB


## Train model

We will fit a random forest with 100 estimators and `max_depth` of 10 to the training set. Zero hyperparameter tuning is done here. If we were to do any hyperparameter tuning, we should use a hold-out validation set.

We train the model on GPU and evaluate on CPU. We evaluate the model using the [F1 score](https://en.wikipedia.org/wiki/F-score).

In [7]:
%%time
progress('start-rf-rapids-dask-fit')

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, ignore_empty_partitions=True)

rfc.fit(taxi_train[features], taxi_train[target_col])
progress('finished-rf-rapids-dask-fit')

CPU times: user 257 ms, sys: 3.12 ms, total: 261 ms
Wall time: 21.9 s


In [8]:
%%time
# Compute F1 
# This is (relatively) slow since we are copying data to the CPU to compute the metric.

preds = rfc.predict_proba(taxi_train[features])[1]
print(f'F1: {f1_score(taxi_train[target_col].compute().to_array(), preds.round().compute().to_array())}')

F1: 0.6681650475249482
CPU times: user 3.87 s, sys: 307 ms, total: 4.17 s
Wall time: 18.3 s


## Evaluate on test set

The test window is all of February 2020 and also accessible via public s3 bucket. The F1 scores are similar between train and test sets.

In [9]:
taxi_feb = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2020-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

taxi_test = preprocess(taxi_feb, target_col=target_col)

In [10]:
# Compute F1 on test set
# This is slow since we are copying data to the CPU to compute the metric.

preds = rfc.predict_proba(taxi_test[features])[1]
print(f'F1: {f1_score(taxi_test[target_col].compute().to_array(), preds.round().compute().to_array())}')

F1: 0.6658098920024954


## Simulate "live" inference on March

As every new batch of points comes in, we make a prediction. We compute the rolling (F1 score since March 1) and daily F1 scores. Note that the daily F1 score drops significantly, but this performance degradation is not so pronounced if we just monitor the rolling F1 score.

In [11]:
# First, load and sort the march dataframe

taxi_march = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2020-03.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

taxi_inference = preprocess(taxi_march, target_col=target_col, start_date='2020-03-01', end_date='2020-03-31').sort_values(by=['tpep_dropoff_datetime'], ascending=True).reset_index(drop=True)
taxi_inference['day'] = taxi_inference.tpep_dropoff_datetime.dt.day.to_dask_array()

In [12]:
# Save predictions as a new column, compute rolling F1 score

taxi_inference['predicted_prob'] = rfc.predict_proba(taxi_inference[features])[1]
taxi_inference['prediction'] = taxi_inference['predicted_prob'].round().astype('int32')
taxi_inference['rolling_f1'] = f1_streaming(taxi_inference, target_col, 'prediction')
daily_f1 = taxi_inference.groupby('day').apply(get_daily_f1_score, meta={'day': int, 'rolling_f1': float, 'daily_f1': float})

In [13]:
daily_f1.sort_values(by='day', ascending=True).compute()

,day,rolling_f1,daily_f1
178123,1,0.576629,0.576629
370840,2,0.633320,0.677398
592741,3,0.649983,0.675877
821398,4,0.659940,0.684125
1064741,5,0.675841,0.722298
1307013,6,0.682284,0.708181
58517,7,0.668002,0.555498
225439,8,0.659918,0.572543
400352,9,0.660947,0.670717
583448,10,0.661801,0.670428


## Evaluate model on later months

We see the performance drop in March 2020, but what happens for future months?

In [14]:
# Cycle through many test sets

months = ['2020-03', '2020-04', '2020-05', '2020-06']
month_dfs = {}

for month in months:
    
    if month not in month_dfs:
        print(f'Loading month {month} for the first time.')
        df = dask_cudf.read_csv(
            f"s3://nyc-tlc/trip data/yellow_tripdata_{month}.csv",
            parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
            storage_options={"anon": True},
            assume_missing=True,
        )

        df = preprocess(df, target_col=target_col)
        month_dfs[month] = df.copy()
    
    curr_taxi_test = month_dfs[month]
        
    preds = rfc.predict_proba(curr_taxi_test[features])[1]
    print(month)
    print(f'\tF1: {f1_score(curr_taxi_test[target_col].compute().to_array(), preds.round().compute().to_array())}')

Loading month 2020-03 for the first time.
2020-03
	F1: 0.6592796100378214
Loading month 2020-04 for the first time.
2020-04
	F1: 0.5714705472990737
Loading month 2020-05 for the first time.
2020-05
	F1: 0.5530868473460906
Loading month 2020-06 for the first time.
2020-06
	F1: 0.5967621469282887


## Inspect differences between feature values

Maybe the distribution of data shifted. We could try to quantify this using a 2-sided statistical test (Kolmogorov Smirnov in this example).

### Compare January 2020 vs February 2020

This snippet shows that the p values being small doesn't really tell us much, as we get very small p values when comparing January 2020 vs February 2020 even though we know the F1 score was similar. Curse "big data."

In [15]:
statistics = []
p_values = []

for feature in features:
    statistic, p_value = stats.ks_2samp(taxi_train[feature].compute().to_pandas(), taxi_test[feature].compute().to_pandas())
    statistics.append(statistic)
    p_values.append(p_value)

In [16]:
comparison_df = pd.DataFrame(data={'feature': features, 'statistic': statistics, 'p_value': p_values})
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
0,pickup_weekday,0.046196,0.000000e+00
2,work_hours,0.028587,0.000000e+00
6,trip_time,0.017205,0.000000e+00
7,trip_speed,0.035415,0.000000e+00
1,pickup_hour,0.009676,8.610133e-258
5,trip_distance,0.005312,5.266602e-78
8,PULocationID,0.004083,2.994877e-46
9,DOLocationID,0.003132,2.157559e-27
4,passenger_count,0.002947,2.634493e-24
10,RatecodeID,0.002616,3.047481e-19


### Compare January 2020 vs March 2020

These p values are also small, which is good? But if this method in general sends warning alerts all the time, an end user might not trust it.

In [17]:
statistics = []
p_values = []

for feature in features:
    statistic, p_value = stats.ks_2samp(taxi_train[feature].compute().to_pandas(), taxi_inference[feature].compute().to_pandas())
    statistics.append(statistic)
    p_values.append(p_value)

In [18]:
comparison_df = pd.DataFrame(data={'feature': features, 'statistic': statistics, 'p_value': p_values})
comparison_df.sort_values(by='p_value', ascending=True).head(11)

,feature,statistic,p_value
0,pickup_weekday,0.059051,0.000000e+00
1,pickup_hour,0.017536,0.000000e+00
4,passenger_count,0.022485,0.000000e+00
5,trip_distance,0.017913,0.000000e+00
7,trip_speed,0.030289,0.000000e+00
9,DOLocationID,0.013995,0.000000e+00
8,PULocationID,0.013068,3.746619e-302
2,work_hours,0.010840,5.006014e-208
6,trip_time,0.007507,5.385560e-100
10,RatecodeID,0.005615,3.933726e-56
